<a href="https://colab.research.google.com/github/atulghunavat/atul_INFO5731_Spring2020/blob/master/Assignment%203/INFO5731_Assignment_Three_Atul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
import pandas as pd
review_df = pd.read_csv('/content/amazonreviews_cleaned.csv')

In [0]:
#1.1
import nltk
nltk.download('punkt')
from textblob import TextBlob

def get_ngram(num):
  ngram_dict = {}

  for review in review_df['Cleaned']:
    ngram_list = TextBlob(review).ngrams(num)
    if(len(ngram_list) > 0):
      for ngram in ngram_list:
        ngram_key = tuple(ngram)
        if ngram_key in ngram_dict:
          ngram_dict[ngram_key] += 1
        else:
          ngram_dict[ngram_key] = 1
  return(ngram_dict)

ngram_num = 3
trigram_dict = get_ngram(ngram_num)
col_name = 'N-Grams(' + str(ngram_num) +') Frequency'
trigram_df = pd.DataFrame.from_dict(trigram_dict, orient='index', columns=[col_name])
print(trigram_df)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
                        N-Grams(3) Frequency
(bought, comput, look)                     1
(comput, look, comput)                     1
(look, comput, wife)                       1
(comput, wife, ssd)                        1
(wife, ssd, least)                         1
...                                      ...
(brand, new, pretti)                       1
(new, pretti, much)                        1
(pretti, much, give)                       1
(much, give, dell)                         1
(give, dell, point)                        1

[1404 rows x 1 columns]


In [0]:
#1.2
bigram_dict = get_ngram(2)
unigram_dict = get_ngram(1)

bigram_prob = {}
for bigram in bigram_dict:
  word = tuple((bigram[0],))
  countup = bigram_dict[bigram]
  countdwn = unigram_dict[word]
  prob = countup / countdwn
  bigram_prob[bigram] = prob
# print(bigram_prob)

col_name = 'Bigrams Probability'
bigram_df = pd.DataFrame.from_dict(bigram_prob, orient='index', columns=[col_name])
print(bigram_df)

                  Bigrams Probability
(bought, comput)             0.125000
(comput, look)               0.055556
(look, comput)               0.111111
(comput, wife)               0.055556
(wife, ssd)                  0.333333
...                               ...
(new, pretti)                0.100000
(pretti, much)               0.333333
(much, give)                 1.000000
(give, dell)                 0.333333
(dell, point)                0.041667

[1427 rows x 1 columns]


In [0]:
#1.3
import nltk
import math
nltk.download('brown')

np_dict_freq = {}
for review in review_df['Cleaned']:
  nps = (TextBlob(review)).noun_phrases
  for np in nps:
    if np in np_dict_freq:
      np_dict_freq[np] += 1
    else:
      np_dict_freq[np] = 1
max_freq = max({freq for freq in np_dict_freq.values()})

all_np_prob = []
for review in review_df['Cleaned']:
  each_row_pb = {}
  each_nps = (TextBlob(review)).noun_phrases
  for each_np in each_nps:
    if each_np in each_row_pb:
      each_row_pb[each_np] += 1
    else:
      each_row_pb[each_np] = 1
  for k, v in each_row_pb.items():
    value = v/max_freq
    each_row_pb[k] = value
  all_np_prob.append(each_row_pb)

row_index = [n for n in range(1, len(all_np_prob)+1)]

prob_df = pd.DataFrame (all_np_prob, index=row_index)
prob_df.fillna(0, inplace=True)
prob_df

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


,comput look comput wife ssd,gb ram,comfort work windowsthi,good balanc price spec look lot,littl fragil mac,full keyboard includ number keypadth touchscreen thing,laptop someth,useless doesnt,modern cellphon doesw look laptop ssd,intern hdd tb use store data critic access someth,memori intens,need upgrad extrem happi machin,box boot speed,ssd time bootup complet,run data heavi node applic heavi,server vscode spotifi tab googl chrome thing isnt,program im,bare util cpu ton research absolut,heartbeatonli flaw screen okay wasnt,machin screen,laptop price rang,displayswhi thing touchscreen liter need,laptop person home use mainli internet search surf use microsoft offic mani hour machin,extra ram ssd,fast boot speedi,quiet oper overal machin,quick run,neg dell,loud fan cycl optic drive wonder chip,fan run,poorli pack usabl screen,minut error messag,hard drive need scan error scan complet show multipl system error,replac day,return approv,process daughter need laptop school noth,purchas laptop collegebound engin student price good,dell anoth inspiron ssd,hard drive surpris,loud fan,...,colleg work,nice touchscreen,person useit everyth look laptop,nice laptop everyth need christma gift,batteri run hour,fulli charg,titl descript differ,actual productless memori,hard drive,xpsrecipi amaz speed,fast startup,nice screen qualiti everyth,thing didnt,screw screw complet,satisfi purchas,good laptop high,happi product,fast perfect comput,great product,load ram sata drive,digit password,boom load backlit god,super slim lightweight laptop troubl,cach chrome noth,page load touch,whole thing,problem anymor,thing run background,open set click privaci click background app choos app run background section,toggl switch app,thing sometm hour day joy happi seller,mine costco,super easi return fan louder airplan,overlook didnt,time comput,soooo sloooowwww year,old comput kid run program load page,piec junk brand,new pretti,dell point
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
# Write your code here
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []
for sent in review_df['Cleaned']:
  corpus.append(sent)

vectorizer = TfidfVectorizer(input=corpus, min_df=0.1, stop_words='english')
matrix = vectorizer.fit_transform(corpus)
feature = vectorizer.get_feature_names()
index=[n for n in range(1, len(corpus)+1)]

tfidf_df=pd.DataFrame(matrix.T.todense(), index=feature, columns=index)
tfidf_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
comput,0.459094,0.000000,0.000000,0.000000,0.000000,0.365707,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.536231,0.0,1.0,0.0,0.000000,0.575721,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.474008,1.0,0.687611,0.000000,0.392922,0.000000,0.000000,0.000000,0.697663,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.754236,0.0,0.0,0.000000,0.729386
dell,0.000000,0.000000,0.246344,0.000000,0.312162,0.000000,0.484710,0.000000,0.716351,0.469082,0.000000,0.0,0.0,0.453551,0.491619,0.539548,0.000000,0.0,0.574308,0.393042,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.427442,0.000000,0.162441,0.0,0.0,0.0,0.347488,0.000000,0.000000,0.327824,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.616599,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.331430
drive,0.000000,0.000000,0.304168,0.392367,0.385437,0.410366,0.598486,0.883203,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.411639,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.636486,0.000000
fan,0.000000,0.000000,0.629066,0.000000,0.398571,0.000000,0.309440,0.000000,0.685982,0.000000,0.000000,0.0,0.0,0.579098,0.627703,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.423047,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.423172
far,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.304433,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.344449,0.000000,0.0,0.000000,0.000000,0.000000,0.326273,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.207406,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.596510,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.639982,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
fast,0.000000,0.000000,0.471957,0.000000,0.000000,0.318368,0.232158,0.228401,0.000000,0.000000,0.552183,0.0,0.0,0.217234,0.000000,0.258423,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.317391,0.000000,0.000000,0.155606,0.0,0.0,0.0,0.000000,0.501196,0.000000,0.000000,0.000000,0.691751,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.780807,0.0,0.0,0.412650,0.0,0.000000,0.000000,0.000000,0.582788,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.636122,0.0,0.0,1.0,0.0,0.497885,0.0,0.0,0.656603,0.0,0.0,0.493796,0.000000
good,0.235721,0.000000,0.000000,0.000000,0.352730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.374390,0.000000,0.353964,0.183551,0.0,0.0,0.0,0.000000,0.000000,0.677203,0.370427,0.000000,0.000000,0.000000,1

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

query = 'Dell is the high performing laptop on Amazon.'
cleaned_query = ''
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
regex = re.compile('[^a-zA-Z\s]')

#This method removes numbers, punctuation and special characters.
def remove_noise_special_numbers(text):
    return re.sub(regex, '', text)

#This menthod removes stop words
def remove_stopwords(text):
  words = " ".join([word for word in text.split() if word not in stopwords.words('english')])
  return words

#This method performs stemming
def perform_stemming(text):
  stem = " ".join([stemmer.stem(i) for i in text.split()])
  return stem

#This method performs lemmitization
def perform_lemmatization(text):
  lem = " ".join([lemmatizer.lemmatize(i) for i in text.split()])
  return lem


cleaned_query = remove_noise_special_numbers(query)
cleaned_query = cleaned_query.lower()
cleaned_query = remove_stopwords(cleaned_query)
cleaned_query = perform_stemming(cleaned_query)
cleaned_query = perform_lemmatization(cleaned_query)
print('Cleaned Query: ' + cleaned_query)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Cleaned Query: dell high perform laptop amazon


In [0]:
# Calculate cosine similarity of the query to others documents by using SKLEARN
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


corpus = []
for sent in review_df['Cleaned']:
  corpus.append(sent)

corpus.append(cleaned_query)
vectorizer = TfidfVectorizer(input=corpus, min_df=0.1, stop_words='english')
matrix = vectorizer.fit_transform(corpus)

cosine_dict = {}
for i in range(0, matrix.shape[0]):
  cosim=cosine_similarity(matrix[80], matrix[i])
  cosine_dict[str(i)] = cosim[0][0]
  i+=1


sorted_keys = (sorted(cosine_dict, key=cosine_dict.get,reverse=True))
sorted_cosine_dict = {}
for key in sorted_keys:
  sorted_cosine_dict[key] = cosine_dict[key];

review = []
cosine = []
for skey in sorted_cosine_dict:
  review.append(corpus[int(skey)])
  cosine.append(sorted_cosine_dict[skey])
rank_df['Review'] = review
rank_df['Rank'] = [i for i in range(1, len(sorted_keys)+1)]
rank_df['CosineSimilarity'] = cosine
rank_df.head(45)


,Review,Rank,CosineSimilarity
0,dell high perform laptop amazon,1,1.000000
1,scam money dell dishonest incompet laptop pc m...,2,0.902481
2,look replac year old laptop seen better daysi ...,3,0.847266
3,laptop use wife replac older dell inspiron lap...,4,0.834126
4,liter thing would say laptop wish touchscreen ...,5,0.820241
5,dell product top heep,6,0.812114
6,dell best amaz one isnt even xpsrecipi amaz speed,7,0.812114
7,bootup quit fast exce last laptop hand dont us...,8,0.793357
8,laptop feel cheap key super small backlight ma...,9,0.769501
9,laptop randomli shut would restart work fine o...,10,0.741126


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/atulghunavat/atul_INFO5731_Spring2020/blob/master/amazonreviews_labled_sentiments.csv